In [1]:
%config Completer.use_jedi = False

import configparser
config = configparser.ConfigParser()
config.read('agg-config.txt')


time_step = int(config.get('Configuration', 'time_step'))
time_step2 = int(config.get('Configuration', 'time_step2'))

window_size = int(config.get('Configuration', 'window_size'))
processed_data_folder = config.get('Configuration', 'processed_data_folder')
raw_data_folder = config.get('Configuration', 'raw_data_folder')

ini_date = config.get('Configuration', 'ini_date')
end_date = config.get('Configuration', 'end_date')


activities=config.get('Configuration', 'activities')
activities = [(item.strip()) for item in activities.split(',')]

cross_days=config.get('Configuration', 'cross_days')
cross_days = [(item.strip()) for item in cross_days.split(',')]

priority_activity=config.get('Configuration', 'priority_activity')
priority_activity = [(item.strip()) for item in priority_activity.split(',')]


sensors=config.get('Configuration', 'sensors')
sensors = [(item.strip()) for item in sensors.split(',')]

users_day={}
for cross_day in cross_days:
    users_day[cross_day]=config.get('Configuration', cross_day)
    users_day[cross_day] = [(item.strip()) for item in users_day[cross_day].split(',')]
print(users_day)

{'2024-07-26': ['16fe', '5b66', 'ed9c'], '2024-07-27': ['16fe', '5b66', 'ed9c'], '2024-07-28': ['16fe', '5b66', 'ed9c'], '2024-07-29': ['16fe', '5b66', 'ed9c'], '2024-07-30': ['16fe', '5b66', 'ed9c'], '2024-07-31': ['16fe', '5b66', 'ed9c']}


In [2]:
import numpy as np
from pandas import read_csv    
import pandas as pd
import time

from datetime import datetime

off_zone=60*60*2

def day_time(t0,tN,ti):
    return int((int)((ti+off_zone)/(60*60*24)))


def time2str(t0,tN,tt):
    return datetime.fromtimestamp(tt).strftime("%Y-%m-%d %H:%M:%S")


def day_time0(t0,tN,ti):
    return day_time(ti)-day_time(t0)+1

def relT(t0,tN,ti):
    return (int)((ti-t0)/time_step2)

def sizeT(t0,tN):
    
    return relT(t0,tN,tN)+1
    
def absT(t0,tN,tt):
    return (int)(tt*time_step2+t0)

def initTime(ini_date,end_date):
    tN = (int)(datetime.strptime(end_date, '%Y-%m-%d %H:%M:%S').timestamp())

    t0 = (int)(datetime.strptime(ini_date, '%Y-%m-%d %H:%M:%S').timestamp())
    print (ini_date,"Init date is", t0, "day:",day_time(t0,tN,t0), time2str(t0,tN,t0))
    print (end_date,"End date is", tN, "day:",day_time(t0,tN,tN), time2str(t0,tN,tN))

    day=list(range(day_time(t0,tN,t0),day_time(t0,tN,tN)+1))[0]
    return day,t0,tN
    
initTime(ini_date,end_date)



def getStrDatefrom(day_number):
    # Reference date (1970-01-01)
    reference_date = datetime(1970, 1, 1)
    
    # Compute the date corresponding to the day number
    resulting_date = reference_date + timedelta(days=day_number)
    
    return resulting_date.strftime("%Y-%m-%d")


2024-07-31 02:00:00 Init date is 1722384000 day: 19935 2024-07-31 02:00:00
2024-08-01 01:59:59 End date is 1722470399 day: 19936 2024-08-01 01:59:59


In [3]:
from datetime import datetime, timedelta

days=[]
t0N_days={}

for cross_day in cross_days:
    print(cross_day)
    # Convertir las fechas a objetos datetime
    ini_date_dt = datetime.strptime(ini_date, "%Y-%m-%d %H:%M:%S")
    end_date_dt = datetime.strptime(end_date, "%Y-%m-%d %H:%M:%S")
    cross_day_dt = datetime.strptime(cross_day, "%Y-%m-%d")

    # Reemplazar el día manteniendo la hora original
    new_ini_date = cross_day_dt.replace(hour=ini_date_dt.hour, minute=ini_date_dt.minute, second=ini_date_dt.second)
    new_end_date = cross_day_dt.replace(hour=end_date_dt.hour, minute=end_date_dt.minute, second=end_date_dt.second) + timedelta(days=1)

    # Convertir las fechas de nuevo a cadena si es necesario
    new_ini_date_str = new_ini_date.strftime("%Y-%m-%d %H:%M:%S")
    new_end_date_str = new_end_date.strftime("%Y-%m-%d %H:%M:%S")

    print(f"New ini_date: {new_ini_date_str}")
    print(f"New end_date: {new_end_date_str}")
    day,t0,tN=initTime(new_ini_date_str,new_end_date_str)
    days.append(day)
    t0N_days[day]=(t0,tN)
    

print(days)
print(t0N_days)



2024-07-26
New ini_date: 2024-07-26 02:00:00
New end_date: 2024-07-27 01:59:59
2024-07-26 02:00:00 Init date is 1721952000 day: 19930 2024-07-26 02:00:00
2024-07-27 01:59:59 End date is 1722038399 day: 19931 2024-07-27 01:59:59
2024-07-27
New ini_date: 2024-07-27 02:00:00
New end_date: 2024-07-28 01:59:59
2024-07-27 02:00:00 Init date is 1722038400 day: 19931 2024-07-27 02:00:00
2024-07-28 01:59:59 End date is 1722124799 day: 19932 2024-07-28 01:59:59
2024-07-28
New ini_date: 2024-07-28 02:00:00
New end_date: 2024-07-29 01:59:59
2024-07-28 02:00:00 Init date is 1722124800 day: 19932 2024-07-28 02:00:00
2024-07-29 01:59:59 End date is 1722211199 day: 19933 2024-07-29 01:59:59
2024-07-29
New ini_date: 2024-07-29 02:00:00
New end_date: 2024-07-30 01:59:59
2024-07-29 02:00:00 Init date is 1722211200 day: 19933 2024-07-29 02:00:00
2024-07-30 01:59:59 End date is 1722297599 day: 19934 2024-07-30 01:59:59
2024-07-30
New ini_date: 2024-07-30 02:00:00
New end_date: 2024-07-31 01:59:59
2024-07-3

In [4]:

user_activity=np.zeros((len(activities),3,len(days),sizeT(t0,tN)))
for dx,day in enumerate(days):
    day_str=getStrDatefrom(day)
    users=users_day[day_str]
    print(day,users)
    for ux,user in enumerate(users):
        for ax,activity in enumerate(activities):
            df_total=read_csv(processed_data_folder+"/DAY_"+str(day)+"/act/"+activity+"."+user+".tsv", sep="\t",parse_dates=True, header=None, names=["ix","d0","dN", "degree"])
            print(df_total)
            print(pd.to_datetime(df_total['d0']))
            df_total['d0'] = pd.to_datetime(df_total['d0'])
            df_total['dN'] = pd.to_datetime(df_total['dN'])
            print(df_total)

            (t0,tN)=t0N_days[day]
            ts=list(range(t0,tN,time_step2))
            for tx,tt in enumerate(ts):
                ttt=pd.to_datetime(time2str(t0,tN,tt))
#                print(ttt)
                row = df_total[(df_total['d0'] <= ttt) & (df_total['dN'] >= ttt)]
#                print("df_in#tervalo",row)
                if(row.size>0):
                    act=activity
                else:
                    act="IDLE"
#                print("\tlabel:",act)
                if(act == activity):
                    user_activity[ax][ux][dx][tx]=1
                else:
                    user_activity[ax][ux][dx][tx]=0





19930 ['16fe', '5b66', 'ed9c']
             ix                   d0                   dN  degree
0    (304, 305)  2024-07-26 07:04:00  2024-07-26 07:05:00    0.36
1    (345, 345)  2024-07-26 07:45:00  2024-07-26 07:45:00    0.31
2    (765, 765)  2024-07-26 14:45:00  2024-07-26 14:45:00    0.34
3    (775, 785)  2024-07-26 14:55:00  2024-07-26 15:05:00    0.61
4  (1082, 1082)  2024-07-26 20:02:00  2024-07-26 20:02:00    0.25
5  (1185, 1186)  2024-07-26 21:45:00  2024-07-26 21:46:00    0.22
6  (1292, 1292)  2024-07-26 23:32:00  2024-07-26 23:32:00    0.24
7  (1372, 1372)  2024-07-27 00:52:00  2024-07-27 00:52:00    0.39
0   2024-07-26 07:04:00
1   2024-07-26 07:45:00
2   2024-07-26 14:45:00
3   2024-07-26 14:55:00
4   2024-07-26 20:02:00
5   2024-07-26 21:45:00
6   2024-07-26 23:32:00
7   2024-07-27 00:52:00
Name: d0, dtype: datetime64[ns]
             ix                  d0                  dN  degree
0    (304, 305) 2024-07-26 07:04:00 2024-07-26 07:05:00    0.36
1    (345, 345) 2024-07

           ix                   d0                   dN  degree
0  (763, 763)  2024-07-26 14:43:00  2024-07-26 14:43:00    0.58
0   2024-07-26 14:43:00
Name: d0, dtype: datetime64[ns]
           ix                  d0                  dN  degree
0  (763, 763) 2024-07-26 14:43:00 2024-07-26 14:43:00    0.58


             ix                   d0                   dN  degree
0    (303, 304)  2024-07-26 07:03:00  2024-07-26 07:04:00    0.87
1    (758, 759)  2024-07-26 14:38:00  2024-07-26 14:39:00    0.42
2    (907, 908)  2024-07-26 17:07:00  2024-07-26 17:08:00    0.96
3  (1097, 1098)  2024-07-26 20:17:00  2024-07-26 20:18:00    0.19
4  (1373, 1373)  2024-07-27 00:53:00  2024-07-27 00:53:00    0.96
0   2024-07-26 07:03:00
1   2024-07-26 14:38:00
2   2024-07-26 17:07:00
3   2024-07-26 20:17:00
4   2024-07-27 00:53:00
Name: d0, dtype: datetime64[ns]
             ix                  d0                  dN  degree
0    (303, 304) 2024-07-26 07:03:00 2024-07-26 07:04:00    0.87
1    (758, 759) 2024-07-26 14:38:00 2024-07-26 14:39:00    0.42
2    (907, 908) 2024-07-26 17:07:00 2024-07-26 17:08:00    0.96
3  (1097, 1098) 2024-07-26 20:17:00 2024-07-26 20:18:00    0.19
4  (1373, 1373) 2024-07-27 00:53:00 2024-07-27 00:53:00    0.96


             ix                   d0                   dN  degree
0  (1305, 1316)  2024-07-26 23:45:00  2024-07-26 23:56:00    0.13
0   2024-07-26 23:45:00
Name: d0, dtype: datetime64[ns]
             ix                  d0                  dN  degree
0  (1305, 1316) 2024-07-26 23:45:00 2024-07-26 23:56:00    0.13


             ix                   d0                   dN  degree
0      (1, 298)  2024-07-26 02:01:00  2024-07-26 06:58:00    0.67
1  (1378, 1438)  2024-07-27 00:58:00  2024-07-27 01:58:00    0.54
0   2024-07-26 02:01:00
1   2024-07-27 00:58:00
Name: d0, dtype: datetime64[ns]
             ix                  d0                  dN  degree
0      (1, 298) 2024-07-26 02:01:00 2024-07-26 06:58:00    0.67
1  (1378, 1438) 2024-07-27 00:58:00 2024-07-27 01:58:00    0.54


...

            ix                   d0                   dN  degree
0  (986, 1074)  2024-07-31 18:26:00  2024-07-31 19:54:00    0.86
0   2024-07-31 18:26:00
Name: d0, dtype: datetime64[ns]
            ix                  d0                  dN  degree
0  (986, 1074) 2024-07-31 18:26:00 2024-07-31 19:54:00    0.86


             ix                   d0                   dN  degree
0      (1, 287)  2024-07-31 02:01:00  2024-07-31 06:47:00    0.86
1    (799, 964)  2024-07-31 15:19:00  2024-07-31 18:04:00    0.76
2  (1430, 1438)  2024-08-01 01:50:00  2024-08-01 01:58:00    0.25
0   2024-07-31 02:01:00
1   2024-07-31 15:19:00
2   2024-08-01 01:50:00
Name: d0, dtype: datetime64[ns]
             ix                  d0                  dN  degree
0      (1, 287) 2024-07-31 02:01:00 2024-07-31 06:47:00    0.86
1    (799, 964) 2024-07-31 15:19:00 2024-07-31 18:04:00    0.76
2  (1430, 1438) 2024-08-01 01:50:00 2024-08-01 01:58:00    0.25


             ix                   d0                   dN  degree
0    (520, 522)  2024-07-31 10:40:00  2024-07-31 10:42:00    0.17
1    (761, 790)  2024-07-31 14:41:00  2024-07-31 15:10:00    0.71
2  (1076, 1076)  2024-07-31 19:56:00  2024-07-31 19:56:00    0.14
3  (1301, 1301)  2024-07-31 23:41:00  2024-07-31 23:41:00    0.15
0   2024-07-31 10:40:00
1   2024-07-31 14:41:00
2   2024-07-31 19:56:00
3   2024-07-31 23:41:00
Name: d0, dtype: datetime64[ns]
             ix                  d0                  dN  degree
0    (520, 522) 2024-07-31 10:40:00 2024-07-31 10:42:00    0.17
1    (761, 790) 2024-07-31 14:41:00 2024-07-31 15:10:00    0.71
2  (1076, 1076) 2024-07-31 19:56:00 2024-07-31 19:56:00    0.14
3  (1301, 1301) 2024-07-31 23:41:00 2024-07-31 23:41:00    0.15


           ix                   d0                   dN  degree
0  (554, 566)  2024-07-31 11:14:00  2024-07-31 11:26:00    0.16
0   2024-07-31 11:14:00
Name: d0, dtype: datetime64[ns]
           ix                  d0                  dN  degree
0  (554, 566) 2024-07-31 11:14:00 2024-07-31 11:26:00    0.16


             ix                   d0                   dN  degree
0    (316, 722)  2024-07-31 07:16:00  2024-07-31 14:02:00    0.85
1  (1085, 1418)  2024-07-31 20:05:00  2024-08-01 01:38:00    0.86
0   2024-07-31 07:16:00
1   2024-07-31 20:05:00
Name: d0, dtype: datetime64[ns]
             ix                  d0                  dN  degree
0    (316, 722) 2024-07-31 07:16:00 2024-07-31 14:02:00    0.85
1  (1085, 1418) 2024-07-31 20:05:00 2024-08-01 01:38:00    0.86


In [5]:
def interval01(s):
    ret = []
    lastV = False
    lastI = -1
    for t in range(0, len(s)):
        if lastV == True and s[t] == False:
            #print("Interval end at index",t,"degree: ", s[t])
            ret.append((lastI, t - 1))
        if lastV == False and s[t] == True:
            #print("Interval start at index",t,"degree: ", s[t])
            lastI = t
        lastV = s[t]
    
    if lastV == True:
        ret.append((lastI, len(s) - 1))
    print(ret)
    return ret

In [6]:


def getIndexAct(activities,act):
    return activities.index(act)

user_activity = user_activity.astype(bool)
user_activity2=user_activity.copy()


for dx,day in enumerate(days):
    day_str=getStrDatefrom(day)
    users=users_day[day_str]
    print(day,users)
    for ux,user in enumerate(users):
        for ax,activity in enumerate(activities):
            (t0,tN)=t0N_days[day]
            ts=list(range(t0,tN,time_step2))
            with open(processed_data_folder+"/DAY_"+str(day)+"/act/LABEL."+user+"."+activity+".tsv", "w") as file:
                for tx,tt in enumerate(ts):
                    ttt=pd.to_datetime(time2str(t0,tN,tt))
                    file.write(time2str(t0,tN,tt)+"\t"+str(user_activity[ax][ux][dx][tx])+"\n")

for dx,day in enumerate(days):
    day_str=getStrDatefrom(day)
    users=users_day[day_str]
    print(day,users)
    for ux,user in enumerate(users):
        for ax,activity in enumerate(activities):
            print(day,user,activity)
            print(interval01(user_activity[ax,ux,dx,:]))


print("##########PRIORITY#########")
for prio1,act in enumerate(reversed(priority_activity)):
    ix=getIndexAct(activities,act)
    print(act,ix)
    
    for prio2,act2 in enumerate(reversed(priority_activity)):
        if(prio2<=prio1):
            continue
        print("\t",act," NOT ",act2)
        jx=getIndexAct(activities,act2)
        for dx,day in enumerate(days):
            for ux,user in enumerate(users):
                user_activity2[jx,ux,dx,:]=user_activity2[jx,ux,dx,:] & (1-user_activity2[ix,ux,dx,:])
    
    
user_activity2 = user_activity2.astype(bool)
for dx,day in enumerate(days):
    day_str=getStrDatefrom(day)
    users=users_day[day_str]
    print(day,users)
    for ux,user in enumerate(users):
        for ax,activity in enumerate(activities):
            print(day,user,activity)
            print(interval01(user_activity2[ax,ux,dx,:]))


for dx,day in enumerate(days):
    day_str=getStrDatefrom(day)
    users=users_day[day_str]
    print(day,users)
    for ux,user in enumerate(users):
        for ax,activity in enumerate(activities):
            (t0,tN)=t0N_days[day]
            ts=list(range(t0,tN,time_step2))
            with open(processed_data_folder+"/DAY_"+str(day)+"/act/LABEL2."+user+"."+activity+".tsv", "w") as file:
                for tx,tt in enumerate(ts):
                    ttt=pd.to_datetime(time2str(t0,tN,tt))
                    file.write(time2str(t0,tN,tt)+"\t"+str(user_activity2[ax][ux][dx][tx])+"\n")


19930 ['16fe', '5b66', 'ed9c']


19931 ['16fe', '5b66', 'ed9c']


19932 ['16fe', '5b66', 'ed9c']


19933 ['16fe', '5b66', 'ed9c']


19934 ['16fe', '5b66', 'ed9c']


...

19931 ['16fe', '5b66', 'ed9c']


19932 ['16fe', '5b66', 'ed9c']


19933 ['16fe', '5b66', 'ed9c']


19934 ['16fe', '5b66', 'ed9c']


19935 ['16fe', '5b66', 'ed9c']


In [7]:
user_day_act={}
for dx,day in enumerate(days):    
    
    day_str=getStrDatefrom(day)
    users=users_day[day_str]
    print(day,users)
    
    (t0,tN)=t0N_days[day]
    user_day_act[day]={}
    for ux,user in enumerate(users):
            user_day_act[day][user]={}
            for ax,activity in enumerate(activities):
                print(processed_data_folder+"/DAY_"+str(day)+"/act/INTERVAL."+user+"."+activity+".tsv")
                with open(processed_data_folder+"/DAY_"+str(day)+"/act/INTERVAL."+user+"."+activity+".tsv", "w") as file:
                    for ii in interval01(user_activity2[ax,ux,dx,:]):
                        user_day_act[day][user][ii[0]]=(activity,ii)
                        file.write(str(ii)+"\t"+datetime.fromtimestamp(absT(t0,tN,ii[0])).strftime("%Y-%m-%d %H:%M:%S")+"\t"+datetime.fromtimestamp(absT(t0,tN,ii[1])).strftime("%Y-%m-%d %H:%M:%S")+"\n")
                   #         file0.write(activity+"\t"+str(ii)+"\t"+datetime.fromtimestamp(absT(t0,tN,ii[0])).strftime("%Y-%m-%d %H:%M:%S")+"\t"+datetime.fromtimestamp(absT(t0,tN,ii[1])).strftime("%Y-%m-%d %H:%M:%S")+"\n")
                            

19930 ['16fe', '5b66', 'ed9c']
./data-har//DAY_19930/act/INTERVAL.16fe.cooking.tsv
[(305, 305), (345, 345), (765, 765), (775, 785), (1082, 1082), (1185, 1186), (1292, 1292), (1372, 1372)]
./data-har//DAY_19930/act/INTERVAL.16fe.shower.tsv
[(763, 763)]
./data-har//DAY_19930/act/INTERVAL.16fe.toileting.tsv
[(303, 304), (758, 759), (907, 908), (1097, 1098), (1373, 1373)]
./data-har//DAY_19930/act/INTERVAL.16fe.pc.tsv
[(1305, 1316)]
./data-har//DAY_19930/act/INTERVAL.16fe.sleep.tsv
[(1, 298), (1378, 1438)]
./data-har//DAY_19930/act/INTERVAL.16fe.kitchen.tsv
[(343, 344), (766, 774)]
./data-har//DAY_19930/act/INTERVAL.16fe.resting.tsv
[(311, 339), (788, 906), (909, 1080), (1265, 1265)]
./data-har//DAY_19930/act/INTERVAL.16fe.exit.tsv
[(349, 753), (1107, 1173)]
./data-har//DAY_19930/act/INTERVAL.5b66.cooking.tsv
[(401, 402), (414, 415), (670, 675), (685, 687), (699, 700), (715, 718), (727, 735), (760, 760), (765, 765), (775, 786), (801, 807), (825, 826), (1072, 1072), (1082, 1082), (1150, 115

[(339, 444), (449, 760), (1069, 1073), (1075, 1351)]
./data-har//DAY_19931/act/INTERVAL.5b66.cooking.tsv
[(13, 13), (519, 519), (527, 529), (535, 535), (560, 561), (590, 590), (602, 602), (655, 657), (671, 671), (685, 685), (691, 699), (705, 705), (731, 731), (736, 739), (755, 764), (767, 780), (787, 811), (819, 819), (1055, 1055), (1083, 1085), (1355, 1355), (1373, 1373), (1379, 1379)]
./data-har//DAY_19931/act/INTERVAL.5b66.shower.tsv
[]
./data-har//DAY_19931/act/INTERVAL.5b66.toileting.tsv
[(4, 6), (65, 66), (484, 486), (521, 521), (530, 531), (578, 579), (604, 615), (626, 630), (658, 659), (733, 734), (765, 766), (781, 786), (826, 832), (1060, 1061), (1072, 1078), (1356, 1362), (1389, 1390), (1398, 1399)]
./data-har//DAY_19931/act/INTERVAL.5b66.pc.tsv
[(995, 997)]
./data-har//DAY_19931/act/INTERVAL.5b66.sleep.tsv
[(16, 64), (72, 479)]
./data-har//DAY_19931/act/INTERVAL.5b66.kitchen.tsv
[(1, 3), (7, 12), (14, 15), (67, 71), (520, 520), (522, 526), (532, 534), (536, 559), (562, 577),

./data-har//DAY_19932/act/INTERVAL.ed9c.sleep.tsv
[(1, 5), (11, 261), (264, 265), (1262, 1438)]
./data-har//DAY_19932/act/INTERVAL.ed9c.kitchen.tsv
[(1158, 1158)]
./data-har//DAY_19932/act/INTERVAL.ed9c.resting.tsv
[(1186, 1214)]
./data-har//DAY_19932/act/INTERVAL.ed9c.exit.tsv
[(285, 1151)]
19933 ['16fe', '5b66', 'ed9c']
./data-har//DAY_19933/act/INTERVAL.16fe.cooking.tsv
[(303, 303), (780, 784), (1114, 1114), (1201, 1201), (1260, 1260)]
./data-har//DAY_19933/act/INTERVAL.16fe.shower.tsv
[(759, 759)]
./data-har//DAY_19933/act/INTERVAL.16fe.toileting.tsv
[(214, 215), (301, 302), (754, 755), (763, 764), (790, 790), (945, 947), (1304, 1305)]
./data-har//DAY_19933/act/INTERVAL.16fe.pc.tsv
[(1287, 1290)]
./data-har//DAY_19933/act/INTERVAL.16fe.sleep.tsv
[(8, 213), (216, 296), (1310, 1438)]
./data-har//DAY_19933/act/INTERVAL.16fe.kitchen.tsv
[(757, 758), (760, 762), (765, 779), (785, 786)]
./data-har//DAY_19933/act/INTERVAL.16fe.resting.tsv
[(792, 944), (948, 1107)]
./data-har//DAY_19933/ac

[]
./data-har//DAY_19934/act/INTERVAL.5b66.sleep.tsv
[(20, 373), (378, 475)]
./data-har//DAY_19934/act/INTERVAL.5b66.kitchen.tsv
[(1, 1), (3, 4), (10, 14), (17, 19), (482, 488), (496, 498), (506, 506), (514, 515), (649, 657), (660, 660), (666, 677), (679, 685), (687, 721), (723, 735), (742, 744), (759, 767), (777, 782), (788, 792), (794, 795), (799, 826), (1062, 1063), (1071, 1075), (1077, 1096), (1098, 1115), (1119, 1121), (1124, 1125), (1184, 1191), (1222, 1226), (1231, 1242), (1244, 1249), (1254, 1262), (1266, 1267), (1431, 1433), (1438, 1438)]
./data-har//DAY_19934/act/INTERVAL.5b66.resting.tsv
[(499, 505), (507, 512), (737, 741), (745, 747), (1049, 1060), (1196, 1211), (1268, 1365), (1369, 1426)]
./data-har//DAY_19934/act/INTERVAL.5b66.exit.tsv
[(528, 586), (588, 641), (1129, 1177)]
./data-har//DAY_19934/act/INTERVAL.ed9c.cooking.tsv
[(756, 758), (768, 776), (968, 968), (1360, 1360)]
./data-har//DAY_19934/act/INTERVAL.ed9c.shower.tsv
[(303, 305)]
./data-har//DAY_19934/act/INTERVAL

./data-har//DAY_19935/act/INTERVAL.5b66.toileting.tsv
[(105, 106), (287, 288), (423, 423), (436, 438), (482, 488), (728, 734), (759, 761), (792, 795), (806, 813), (829, 830), (845, 847), (1038, 1039), (1062, 1063), (1073, 1075), (1232, 1235), (1328, 1331), (1425, 1436)]
./data-har//DAY_19935/act/INTERVAL.5b66.pc.tsv
[]
./data-har//DAY_19935/act/INTERVAL.5b66.sleep.tsv
[(10, 104), (107, 286), (289, 418)]
./data-har//DAY_19935/act/INTERVAL.5b66.kitchen.tsv
[(1, 5), (7, 7), (425, 433), (435, 435), (439, 439), (459, 473), (476, 481), (489, 493), (495, 495), (644, 648), (660, 681), (683, 690), (695, 724), (765, 776), (778, 781), (785, 790), (802, 805), (814, 828), (831, 835), (837, 844), (848, 848), (1064, 1069), (1071, 1072), (1076, 1088), (1154, 1156), (1174, 1186), (1204, 1210), (1214, 1231), (1236, 1242), (1245, 1247), (1437, 1438)]
./data-har//DAY_19935/act/INTERVAL.5b66.resting.tsv
[(445, 458), (740, 754), (871, 874), (936, 937), (991, 998), (1159, 1173), (1248, 1327), (1332, 1423)]
.

In [8]:


for dx,day in enumerate(days):

    day_str=getStrDatefrom(day)
    users=users_day[day_str]
    print(day,users)

    (t0,tN)=t0N_days[day]
    for ux,user in enumerate(users):
        sorted_dict = {key: user_day_act[day][user][key] for key in sorted(user_day_act[day][user])}

        print(sorted_dict)
        print(processed_data_folder+"/DAY_"+str(day)+"/INTERVAL."+user+".all.tsv")
        with open(processed_data_folder+"/DAY_"+str(day)+"/INTERVAL."+user+".all.tsv", "w") as file0:
                for act_ii in sorted_dict:
                    activity=sorted_dict[act_ii][0]
                    ii=sorted_dict[act_ii][1]
                    file0.write(activity+"\t\t\t"+str(ii)+"\t\t"+datetime.fromtimestamp(absT(t0,tN,ii[0])).strftime("%Y-%m-%d %H:%M:%S")+"\t"+datetime.fromtimestamp(absT(t0,tN,ii[1])).strftime("%Y-%m-%d %H:%M:%S")+"\n")
                            

19930 ['16fe', '5b66', 'ed9c']
{1: ('sleep', (1, 298)), 303: ('toileting', (303, 304)), 305: ('cooking', (305, 305)), 311: ('resting', (311, 339)), 343: ('kitchen', (343, 344)), 345: ('cooking', (345, 345)), 349: ('exit', (349, 753)), 758: ('toileting', (758, 759)), 763: ('shower', (763, 763)), 765: ('cooking', (765, 765)), 766: ('kitchen', (766, 774)), 775: ('cooking', (775, 785)), 788: ('resting', (788, 906)), 907: ('toileting', (907, 908)), 909: ('resting', (909, 1080)), 1082: ('cooking', (1082, 1082)), 1097: ('toileting', (1097, 1098)), 1107: ('exit', (1107, 1173)), 1185: ('cooking', (1185, 1186)), 1265: ('resting', (1265, 1265)), 1292: ('cooking', (1292, 1292)), 1305: ('pc', (1305, 1316)), 1372: ('cooking', (1372, 1372)), 1373: ('toileting', (1373, 1373)), 1378: ('sleep', (1378, 1438))}
./data-har//DAY_19930/INTERVAL.16fe.all.tsv
{1: ('sleep', (1, 78)), 79: ('toileting', (79, 81)), 82: ('sleep', (82, 392)), 393: ('toileting', (393, 398)), 399: ('kitchen', (399, 400)), 401: ('cooki